<a href="https://colab.research.google.com/github/jazzathoth/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module2-sampling-confidence-intervals-and-hypothesis-testing/LS_DS_142_Sampling_Confidence_Intervals_and_Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 142
## Sampling, Confidence Intervals, and Hypothesis Testing

## Prepare - examine other available hypothesis tests

If you had to pick a single hypothesis test in your toolbox, t-test would probably be the best choice - but the good news is you don't have to pick just one! Here's some of the others to be aware of:

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import chisquare  # One-way chi square test

In [2]:


# Chi square can take any crosstab/table and test the independence of rows/cols
# The null hypothesis is that the rows/cols are independent -> low chi square
# The alternative is that there is a dependence -> high chi square
# Be aware! Chi square does *not* tell you direction/causation

ind_obs = np.array([[1, 1], [2, 2]]).T
print(ind_obs)
print(chisquare(ind_obs, axis=None))

dep_obs = np.array([[16, 18, 16, 14, 12, 12], [32, 24, 16, 28, 20, 24]]).T
print(dep_obs)
print(chisquare(dep_obs, axis=None))

[[1 2]
 [1 2]]
Power_divergenceResult(statistic=0.6666666666666666, pvalue=0.8810148425137847)
[[16 32]
 [18 24]
 [16 16]
 [14 28]
 [12 20]
 [12 24]]
Power_divergenceResult(statistic=23.31034482758621, pvalue=0.015975692534127565)


In [3]:
# Distribution tests:
# We often assume that something is normal, but it can be important to *check*

# For example, later on with predictive modeling, a typical assumption is that
# residuals (prediction errors) are normal - checking is a good diagnostic

from scipy.stats import normaltest
# Poisson models arrival times and is related to the binomial (coinflip)
sample = np.random.poisson(5, 1000)
print(normaltest(sample))  # Pretty clearly not normal

NormaltestResult(statistic=42.378347669716064, pvalue=6.275650186837583e-10)


In [4]:
# Kruskal-Wallis H-test - compare the median rank between 2+ groups
# Can be applied to ranking decisions/outcomes/recommendations
# The underlying math comes from chi-square distribution, and is best for n>5
from scipy.stats import kruskal

x1 = [1, 3, 5, 7, 9]
y1 = [2, 4, 6, 8, 10]
print(kruskal(x1, y1))  # x1 is a little better, but not "significantly" so

x2 = [1, 1, 1]
y2 = [2, 2, 2]
z = [2, 2]  # Hey, a third group, and of different size!
print(kruskal(x2, y2, z))  # x clearly dominates

KruskalResult(statistic=0.2727272727272734, pvalue=0.6015081344405895)
KruskalResult(statistic=7.0, pvalue=0.0301973834223185)


And there's many more! `scipy.stats` is fairly comprehensive, though there are even more available if you delve into the extended world of statistics packages. As tests get increasingly obscure and specialized, the importance of knowing them by heart becomes small - but being able to look them up and figure them out when they *are* relevant is still important.

## Live Lecture - let's explore some more of scipy.stats

Candidate topics to explore:

- `scipy.stats.chi2` - the Chi-squared distribution, which we can use to reproduce the Chi-squared test
- Calculate the Chi-Squared test statistic "by hand" (with code), and feed it into `chi2`
- Build a confidence interval with `stats.t.ppf`, the t-distribution percentile point function (the inverse of the CDF) - we can write a function to return a tuple of `(mean, lower bound, upper bound)` that you can then use for the assignment (visualizing confidence intervals)

In [0]:
# Taking requests! Come to lecture with a topic or problem and we'll try it.

In [6]:
gender = ['male', 'male','male','female','female','female']
eats_outside = ['outside', 'inside', 'inside', 'inside', 'outside', 'outside']

df = pd.DataFrame({'gender': gender, 'preference': eats_outside})
df.head()

,gender,preference
0,male,outside
1,male,inside
2,male,inside
3,female,inside
4,female,outside


In [7]:
table = pd.crosstab(df.gender, df.preference)
table

preference,inside,outside
gender,,
female,1,2
male,2,1


In [8]:
pd.crosstab(df.gender, df.preference, margins=True)

preference,inside,outside,All
gender,,,
female,1,2,3
male,2,1,3
All,3,3,6


In [9]:
df = df.replace('male', 0)
df = df.replace('female', 1)
df = df.replace('outside', 0)
df = df.replace('inside', 1)
df.head()

,gender,preference
0,0,0
1,0,1
2,0,1
3,1,1
4,1,0


In [10]:
pd.crosstab(df.gender, df.preference, margins=True)
expected =  [[1.5, 1.5],
            [1.5, 1.5]]

pd.crosstab(df.gender, df.preference, margins=True, normalize='all')

observed = np.array([[.5, .5],[.5, .5]])

deviation = numerator = observed - expected

print(numerator)

deviation_squared = deviation**2
print('deviation squared:  ', deviation_squared, '\n')

fraction = (deviation_squared / expected)

print('\nfraction:  ', fraction, '\n')

[[-1. -1.]
 [-1. -1.]]
deviation squared:   [[1. 1.]
 [1. 1.]] 


fraction:   [[0.66666667 0.66666667]
 [0.66666667 0.66666667]] 



In [11]:
expected_values = [[1.5, 1.5], [1.5, 1.5]]
deviation = (((.5)**2) / 1.5) * 4  # 0.5^2 deviation per cell, scaled and added
print(deviation)

0.6666666666666666


In [12]:
chi_data =  [[1,2],
            [2,1]]

from scipy.stats import chisquare # one way chi square

chisquare(chi_data, axis=None)

Power_divergenceResult(statistic=0.6666666666666666, pvalue=0.8810148425137847)

In [13]:
from scipy.stats import chi2_contingency

chi2stat, pvalue, dof, observed = chi2_contingency(table)

print('chi2 stat:  ', chi2stat, '\n')
print('p-value:  ', pvalue, '\n')
print('degrees of freedom:  ', dof, '\n')
print('Contingency Table: \n', observed, '\n')

chi2 stat:   0.0 

p-value:   1.0 

degrees of freedom:   1 

Contingency Table: 
 [[1.5 1.5]
 [1.5 1.5]] 



In [14]:
def lazy_chisquare(observed, expected):
  chisquare = 0
  for row_obs, row_exp in zip(observed, expected):
    for obs, exp in zip(row_obs, row_exp):
      chisquare += (obs - exp)**2 / exp
  return chisquare

chi_data = [[1, 2], [2, 1]]
expected_values = [[1.5, 1.5], [1.5, 1.5]]
chistat = lazy_chisquare(chi_data, expected_values)
chistat

0.6666666666666666

# Confidence Intervals

In [15]:
#confidence_interval = [lower_bound, upper_bound]

coinflips = np.random.binomial(n=1, p=.7, size=100)
print(coinflips)

[1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1
 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1
 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1]


In [16]:
import scipy.stats as stats

stats.ttest_1samp(coinflips, 0.5)

Ttest_1sampResult(statistic=6.383694290536714, pvalue=5.617298020139168e-09)

In [0]:
def confidence_interval(data, confidence=.95):
  
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) / 2., n-1)
  return(mean, mean - interval, mean + interval)

def report_confidence_interval(confidence_interval):
  s = "our mean lies in the interval [{:.2}, {:.2}]".format(
         confidence_interval[1], confidence_interval[2])
  return s


In [18]:
coinflip_interval = confidence_interval(coinflips)
coinflip_interval

(0.77, 0.6860770311790257, 0.8539229688209743)

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):

1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)

In [21]:
# TODO - your code!

votes_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
votes = pd.read_csv(votes_url, header=None, names=['party', 
                                                   'infant_bill', 
                                                   'water_project', 
                                                   'budget', 
                                                   'physician_bill', 
                                                   'aid_elsalvador',
                                                   'church_school',
                                                   'anti_satellite',
                                                   'contra_aid',
                                                   'mx_missile',
                                                   'immigration',
                                                   'synfuels',
                                                   'edu_spending',
                                                   'superfund_litigation',
                                                   'crime',
                                                   'duty_free',
                                                   's_africa_export'])

votes.shape

(435, 17)

In [22]:
votes.describe()

,party,infant_bill,water_project,budget,physician_bill,aid_elsalvador,church_school,anti_satellite,contra_aid,mx_missile,immigration,synfuels,edu_spending,superfund_litigation,crime,duty_free,s_africa_export
count,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435
unique,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


In [23]:
votes['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [25]:
votes_nan = pd.DataFrame()

for i in votes:
  votes_nan[i] = votes[i].replace('?', np.nan)
print(votes_nan.isnull().sum())

party                     0
infant_bill              12
water_project            48
budget                   11
physician_bill           11
aid_elsalvador           15
church_school            11
anti_satellite           14
contra_aid               15
mx_missile               22
immigration               7
synfuels                 21
edu_spending             31
superfund_litigation     25
crime                    17
duty_free                28
s_africa_export         104
dtype: int64


In [26]:
democrats = votes.loc[votes['party'] == 'democrat']
republicans = votes.loc[votes['party'] == 'republican']

democrats.shape, republicans.shape

((267, 17), (168, 17))

In [0]:
counts_party = pd.DataFrame({'party vote':['democrats yes', 'democrats no', 'republicans yes', 'republicans no']})

dyes, dno, ryes, rno = int(), int(), int(), int()

for bill in votes_nan.drop(['party'], axis=1):
  for ind in votes_nan[bill].index:

    vote = votes_nan[bill].loc[ind]
    party = votes_nan['party'].loc[ind]

    if party == 'democrat':
      if vote == 'y':
        dyes += 1
      elif vote == 'n':
        dno += 1

    elif party == 'republican':
      if vote == 'y':
        ryes += 1
      elif vote == 'n':
        rno += 1
  
  counts_party[bill] = [dyes, dno, ryes, rno]
  
counts_party.set_index('party vote')
    
#democrats = votes.loc[votes['party'] == 'democrat']
#republicans = votes.loc[votes['party'] == 'republican']

In [48]:
counts_party

,party vote,infant_bill,water_project,budget,physician_bill,aid_elsalvador,church_school,anti_satellite,contra_aid,mx_missile,immigration,synfuels,edu_spending,superfund_litigation,crime,duty_free,s_africa_export
0,democrats yes,156,276,507,521,576,699,899,1117,1305,1429,1558,1594,1667,1757,1917,2090
1,democrats no,102,221,250,495,695,830,889,934,994,1133,1259,1472,1651,1818,1909,1921
2,republicans yes,31,106,128,291,448,597,636,660,679,771,792,927,1063,1221,1235,1331
3,republicans no,134,207,349,351,359,376,499,632,778,851,989,1009,1031,1034,1176,1226


In [0]:
counts_party.hist(figsize=(12,12));


In [0]:
dem_enc = democrats.drop(['party'], axis=1).replace('?', np.nan).replace({'y':1,'n':0})
rep_enc = republicans.drop(['party'], axis=1).replace('?', np.nan).replace({'y':1,'n':0})

vote_enc = votes.drop(['party'], axis=1).replace('?', np.nan).replace({'y':1,'n':0})

In [79]:
dem_enc.isnull().sum()

infant_bill              9
water_project           28
budget                   7
physician_bill           8
aid_elsalvador          12
church_school            9
anti_satellite           8
contra_aid               4
mx_missile              19
immigration              4
synfuels                12
edu_spending            18
superfund_litigation    15
crime                   10
duty_free               16
s_africa_export         82
dtype: int64

In [0]:
dem_total, rep_total, dem_rep = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

for bill in dem_enc:
  res, prob = stats.ttest_ind(dem_enc[bill], vote_enc[bill], nan_policy='omit')
  dem_total[bill] = [res,prob]

for bill in rep_enc:
  res, prob = stats.ttest_ind(vote_enc[bill], rep_enc[bill], nan_policy='omit')
  rep_total[bill] = [res,prob]
  
for bill in rep_enc:
  res, prob = stats.ttest_ind(dem_enc[bill], rep_enc[bill], nan_policy='omit', equal_var=False)
  dem_rep[bill] = [res,prob]
  


In [86]:
dem_total

,infant_bill,water_project,budget,physician_bill,aid_elsalvador,church_school,anti_satellite,contra_aid,mx_missile,immigration,synfuels,edu_spending,superfund_litigation,crime,duty_free,s_africa_export
0,4.162198,-0.043300,8.556534e+00,-1.114779e+01,-7.761465e+00,-4.276984,5.526319e+00,7.091408e+00,6.730152e+00,-0.846544,3.689174,-7.757992e+00,-5.685912e+00,-6.302896e+00,5.336369e+00,3.852854
1,0.000036,0.965476,7.663688e-17,1.260374e-26,3.127602e-14,0.000022,4.666995e-08,3.332421e-12,3.687337e-11,0.397543,0.000243,3.344254e-14,1.954831e-08,5.283423e-10,1.307617e-07,0.000132


In [87]:
rep_total

,infant_bill,water_project,budget,physician_bill,aid_elsalvador,church_school,anti_satellite,contra_aid,mx_missile,immigration,synfuels,edu_spending,superfund_litigation,crime,duty_free,s_africa_export
0,5.891209e+00,-0.059506,1.106577e+01,-1.469305e+01,-1.106411e+01,-6.386796e+00,7.393094e+00,9.788809e+00,9.181555e+00,-1.154858,5.525932e+00,-1.038151e+01,-8.093952e+00,-9.863802e+00,8.015250e+00,3.730057
1,6.467122e-09,0.952572,5.606429e-26,7.587344e-42,5.842524e-26,3.443896e-10,5.030722e-13,4.957020e-21,7.577133e-19,0.248615,4.987868e-08,3.354232e-23,3.548465e-15,2.589970e-21,6.412413e-15,0.000215


In [88]:
dem_rep

,infant_bill,water_project,budget,physician_bill,aid_elsalvador,church_school,anti_satellite,contra_aid,mx_missile,immigration,synfuels,edu_spending,superfund_litigation,crime,duty_free,s_africa_export
0,9.662337e+00,-0.088940,2.279325e+01,-5.670481e+01,-2.390729e+01,-1.076702e+01,1.246749e+01,1.825486e+01,1.740905e+01,-1.737470,9.040574e+00,-2.072482e+01,-1.435257e+01,-1.992766e+01,1.437624e+01,6.397887e+00
1,5.429019e-20,0.929187,1.946704e-69,8.820761e-193,2.898373e-79,4.830932e-24,1.362781e-29,2.288121e-52,4.289262e-51,0.083185,6.298960e-18,3.568751e-62,7.597900e-38,6.862816e-58,3.812922e-38,1.042864e-09


In [89]:
dem_t1, rep_t1 = pd.DataFrame(), pd.DataFrame()

for bill in dem_enc:
  res, prob = stats.ttest_1samp(dem_enc[bill], nan_policy='omit')
  dem_t1[bill] = [res,prob]

for bill in rep_enc:
  res, prob = stats.ttest_ind(rep_enc[bill], nan_policy='omit')
  rep_t1[bill] = [res,prob]
  

TypeError: ignored

In [72]:
tv, prob = stats.ttest_ind(vote_enc['infant_bill'], dem_enc['infant_bill'])
print(tv, prob)
print(type(tv), type(prob))

-4.081601946648043 4.9881980609902794e-05
<class 'numpy.float64'> <class 'numpy.float64'>


In [0]:
import seaborn as sns

,infant_bill,water_project,budget,physician_bill,aid_elsalvador,church_school,anti_satellite,contra_aid,mx_missile,immigration,synfuels,edu_spending,superfund_litigation,crime,duty_free,s_africa_export
2,0,1,1,0,1,1,0,0,0,0,1,0,1,1,0,0
3,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1
4,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1
5,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
6,0,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1
